In [10]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [11]:
# from rs import skill_check

# def test_skill_check(metadata):
#     print(skill_check("sanity", None, metadata))

# metadata = {
#     "character": {"sanity":30},
#     "symptom":[
#         "incoherent speech",
#         "uncontrollable twitching, trembling",
#         "delusions of persecution",
#         "strange appetites (dirt, clay, etc...)",
#         "scratches, punches, and bruises all over the body"
#     ]
# }

# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)

# print(metadata["character"])


In [12]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|normal|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    
    # print(config.metadata["character"])
    
    if "tags" in config.metadata["character"] and "insane" in config.metadata["character"]["tags"]:
        return f"TURN: {count + 1} {is_final} (I'm insane!)"
    
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    # print(config.metadata["character"])
    content = skill_check(skill, difficulty, config.metadata)
    content += f"\n{get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        # config.metadata["memories"] = []
        config.metadata["memories"].append(dict["round summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/ai_gf.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))

The year is 2023, you travelled to some distant mountains and forest in Hokkaido to do some scenic photography in the middle of the week during autumn. You wanted to get away from the city life of Tokyo just so you can make some time for yourself and think about the future and want to do in your life. As you are hiking through the quiet mountain pass, you hear a cry of a girl just in the distance. You quickly made your way towards the sound and found a beautiful young lady on the side of the road, and appears to have accidently injured herself.

What is your response?

possible actions:
1. Offer your assitance and then introduce yourself casually
2. Say a funny pick up line
3. Ignore her and keep walking


I select option 1. TURN: 1 


<start>
Rushing over to her side, you quickly assess the situation before speaking, "Hey, are you okay? That looks painful. Here, let me help you." Your tone is warm and concerned, showing genuine care for her well-being. After helping her to a more comfortable position, you extend a hand, introducing yourself, "I'm Hikaru Mikami. What's your name?"

Mitsuha looks up at you, a mix of surprise and relief flashing across her face. "Thank you, Hikaru. I'm Mitsuha Tomoe. I wasn't watching where I was going and slipped. It's a bit embarrassing," she admits with a sheepish smile. Despite the situation, she seems to appreciate your help and introduction.

possible actions:
1. Ask her if she needs any medical assistance or if there's someone you can call for her.
2. Offer to accompany her to her destination for safety.
3. Make a light-hearted joke to ease the tension.
<end>

I select option 3. TURN: 2 


<start>
With a playful grin, you decide to lighten the mood, "Well, they say every great adventure has its ups and downs. Looks like you've just had a little 'trip' down the adventurous path, huh?" Your tone is light, aiming to bring a smile to her face.

Mitsuha's eyes light up with amusement at your joke, and a gentle laugh escapes her lips. "That's one way to look at it," she responds, her mood visibly lifted by your humor. "I guess I should be more careful on my adventures. Thanks for making me smile, Hikaru."

possible actions:
1. Offer to help her get back on her feet and see if she can walk.
2. Suggest finding a nearby place to rest and assess her injury more thoroughly.
3. Ask her what brought her to this remote area all by herself.
<end>

I select option 1. TURN: 3 


<start>
With a nod, you gently offer your hand to help Mitsuha back to her feet. "Let's see if you can stand. We'll take it slow," you say, ensuring she feels supported and not rushed.

Mitsuha gratefully accepts your help, and with your assistance, she carefully stands. Testing her weight on the injured leg, she winces slightly but manages a small smile. "Looks like I can stand, thanks to you. I might be a bit slow, but I think I can walk."

possible actions:
1. Suggest taking a short break to let her rest before continuing.
2. Offer to walk with her at her pace, ensuring she gets to her destination safely.
3. Ask her again what brought her to this remote area, now that she seems more comfortable.
<end>

I select option 2. TURN: 4 


<start>
"It's no trouble at all. I'll walk with you, at your pace. Just let me know if you need to stop or rest," you offer warmly, showing your willingness to ensure her safety and comfort.

Mitsuha looks genuinely touched by your kindness. "Thank you, Hikaru. That's very kind of you. I'm staying at a small hotel not too far from here. I was trying to capture some of the autumn scenery when I... well, you know," she says with a slight chuckle, referring to her earlier mishap.

As you both start walking slowly, the conversation flows more freely, and you find out that you are actually staying at the same hotel. The coincidence seems to amuse Mitsuha, and the atmosphere between you becomes more relaxed and friendly.

possible actions:
1. Express your surprise and delight at the coincidence, suggesting you could explore the area together.
2. Offer to help her with her photography, showing interest in her hobby.
3. Share a bit about why you're here, opening u

I select option 1. TURN: 5 


<start>
Discovering you're staying at the same hotel, you can't help but express your surprise and delight. "What a coincidence! We're both staying at the same place. Maybe the universe is telling us we should explore some of these scenic spots together," you suggest with a hopeful tone, intrigued by the idea of sharing the experience with someone as spirited as Mitsuha.

Mitsuha's eyes sparkle with excitement at the suggestion. "That sounds like a wonderful idea, Hikaru! I'd love to have someone to share the adventure with, especially after today's...excitement," she says, her voice filled with enthusiasm. "It's settled then. We'll make plans over dinner at the hotel?"

Your offer not only ensures her safety but also opens the door to a new friendship, or perhaps something more, as you both look forward to exploring the beautiful autumn landscape together.

**Summary of the round**: Hikaru found Mitsuha injured on a mountain path and offered his assistanc